# Data Preprocessing

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import os
import pandas as pd
import io

#DATASET_DIR = './'
SAVE_DIR = './'
X = pd.read_csv(io.BytesIO(uploaded['training_set_rel3.tsv']), sep='\t', encoding='ISO-8859-1')
y = X['domain1_score']
X = X.dropna(axis=1)
X = X.drop(columns=['rater1_domain1', 'rater2_domain1', 'domain1_score'])

X

,essay_id,essay_set,essay
0,1,1,"Dear local newspaper, I think effects computer..."
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu..."
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl..."
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that..."
4,5,1,"Dear @LOCATION1, I know having computers has a..."
...,...,...,...
12971,21626,8,In most stories mothers and daughters are eit...
12972,21628,8,I never understood the meaning laughter is th...
12973,21629,8,"When you laugh, is @CAPS5 out of habit, or is ..."
12974,21630,8,Trippin' on fen...


# Training Model

In [ ]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec

def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float64")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model.wv[word])        
    if num_words > 0:
      featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float64")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [ ]:
!python

Python 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import nltk
>>> nltk.download("popular")
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nl

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score
import nltk

# Initializing variables for word2vec model.
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

results = []
y_pred_list = []

count = 1

X_test, X_train, y_test, y_train = train_test_split(X, y, test_size=0.33, random_state=42)

train_essays = X_train['essay']
test_essays = X_test['essay']

sentences = []

for essay in train_essays:
        # Obtaining all sentences from the training essays.
        sentences += essay_to_sentences(essay, remove_stopwords = True)


In [ ]:
print("Training Word2Vec Model...")
model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

clean_train_essays = []

# Generate training and testing data word vectors.
for essay_v in train_essays:
    clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))
trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)

clean_test_essays = []
for essay_v in test_essays:
    clean_test_essays.append(essay_to_wordlist( essay_v, remove_stopwords=True ))
testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )

trainDataVecs = np.array(trainDataVecs)
testDataVecs = np.array(testDataVecs)


Training Word2Vec Model...


In [ ]:
# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))

In [ ]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
from tensorflow.keras.activations import tanh, softmax
from keras.layers import Layer
import keras.backend as K
import numpy as np
import tensorflow as tf

class Attention(Layer):
    def __init__(self,**kwargs):
        super(Attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(Attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return output
        #return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(Attention,self).get_config()

In [ ]:
from keras.utils.vis_utils import plot_model

inputs=tf.keras.layers.Input((1,300))
att_in=Attention()(inputs)
att_out=LSTM(300, input_shape=(1, 300), dropout=0.4, recurrent_dropout=0.4,  return_sequences=True)(att_in)
att_out = LSTM(64, recurrent_dropout=0.4)(att_out)
att_out = Dropout(0.5)(att_out)
outputs = Dense(1, activation='relu')(att_out)

model=tf.keras.Model(inputs,outputs)
model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 300)]          0         
                                                                 
 attention (Attention)       (None, 1, 300)            301       
                                                                 
 lstm (LSTM)                 (None, 1, 300)            721200    
                                                                 
 lstm_1 (LSTM)               (None, 64)                93440     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 815,006
Trainable params: 815,006
Non-trainable

In [ ]:
history = model.fit(trainDataVecs, y_train, batch_size=64, epochs=500)
y_pred = model.predict(testDataVecs)

Epoch 1/500
67/67 [==============================] - 8s 31ms/step - loss: 78.3518 - mae: 4.7372
Epoch 2/500
67/67 [==============================] - 2s 30ms/step - loss: 49.3441 - mae: 3.7178
Epoch 3/500
67/67 [==============================] - 2s 31ms/step - loss: 41.8638 - mae: 3.5715
Epoch 4/500
67/67 [==============================] - 2s 30ms/step - loss: 38.1044 - mae: 3.4782
Epoch 5/500
67/67 [==============================] - 2s 30ms/step - loss: 34.7592 - mae: 3.4019
Epoch 6/500
67/67 [==============================] - 2s 29ms/step - loss: 32.2327 - mae: 3.3302
Epoch 7/500
67/67 [==============================] - 2s 30ms/step - loss: 29.9562 - mae: 3.2314
Epoch 8/500
67/67 [==============================] - 2s 30ms/step - loss: 28.8055 - mae: 3.1350
Epoch 9/500
67/67 [==============================] - 2s 30ms/step - loss: 27.5559 - mae: 3.0268
Epoch 10/500
67/67 [==============================] - 2s 30ms/step - loss: 26.4688 - mae: 2.9530
Epoch 11/500
67/67 [===================

In [ ]:
# Round y_pred to the nearest integer.
y_pred = np.around(y_pred)

# Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
print("Kappa Score: {}".format(result))

Kappa Score: 0.962082401564485
